##### Copyright 2020 The Cirq Developers

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Protocols

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.example.org/cirq/protocols"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on QuantumLib</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/quantumlib/Cirq/blob/master/docs/protocols.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/quantumlib/Cirq/blob/master/docs/protocols.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/Cirq/docs/protocols.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

In [ ]:
try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq
    print("installed cirq.")

# Introduction

Cirq's protocols are very similar concept to Python's built-in protocols that were introduced in [PEP 544](https://www.python.org/dev/peps/pep-0544/).
Python's built-in protocols are extremely convenient, for example behind all the for loops and list comprehensions you can find the Iterator protocol.
As long as an object has the `__iter__()` magic method that returns an iterator object, it has iterator support.
An iterator object has to define `__iter__()` and `__next__()` magic methods, that defines the iterator protocol.
The `iter(val)` builtin function returns an iterator for `val` if it defines the above methods, otherwise throws a `TypeError`. Cirq protocols work similarly.

A canonical Cirq protocol example is the `unitary` protocol that allows to check the unitary matrix of values that support the protocol by calling `cirq.unitary(val)`.

In [ ]:
import cirq 

print(cirq.X)
print("cirq.X unitary:\n", cirq.unitary(cirq.X))

a, b = cirq.LineQubit.range(2)
circuit = cirq.Circuit(cirq.X(a), cirq.Y(b))
print(circuit)
print("circuit unitary:\n", cirq.unitary(circuit))


When an object does not support a given protocol, an error is thrown.

In [ ]:
try: 
    print(cirq.unitary(a)) ## error!
except Exception as e: 
    print("As expected, a qubit does not have a unitary. The error: ")
    print(e)
    

## What is a protocol? 

A protocol is a combination of the following two items: 
- a `SupportsXYZ` class, which defines and documents all the magic functions that need to be implemented in order to support that given protocol 
- the entrypoint function(s), which are exposed to the main cirq namespace as `cirq.xyz()`

Note: While the protcol is technically both of these things, we refer to the public utility functions interchangeably as protocols. See the list of them below.


## Cirq's protocols

Cirq's protocols can be useful in multiple feature areas of quantum computing. Here's a list of all of them. 

| Protocol | Areas | 
|----------|-------|
|`cirq.act_on`| simulation |
|`cirq.channel`| simulation |
|`cirq.apply_channel`| simulation |
|`cirq.has_channel`| simulation |
|`cirq.apply_mixture`| simulation |
|`cirq.mixture`|simulation  |
|`cirq.has_mixture`| simulation |
|`cirq.unitary`| simulation |
|`cirq.apply_unitary`| simulation |
|`cirq.has_unitary`| simulation |
|`cirq.has_stabilizer_effect`| simulation |
|`cirq.trace_distance_bound`| compilation |
|`cirq.equal_up_to_global_phase`| debugging, simulation, compilation |
|`cirq.approx_eq`| debugging, simulation, compilation |
|`cirq.measurement_key`| simulation, analysis   |
|`cirq.qid_shape`| circuit-construction, debugging, simulation |
|`cirq.resolve_parameters`| simulation, compilation, debugging |
|`cirq.parameter_names`| simulation, compilation, debugging |
|`cirq.parameter_symbols`| simulation, compilation, debugging |
|`cirq.commutes`| debugging, compilation, simulation |
|`cirq.decompose`|  debugging, compilation, simulation |
|`cirq.pauli_expansion`|  debugging, compilation, simulation |
|`cirq.inverse`| circuit-construction  |
|`cirq.phase`|  circuit-construction|
|`cirq.pow`| circuit-construction  |
|`cirq.mul`| circuit-construction |
|`cirq.circuit_diagram_info`| visualization |
|`cirq.qasm`| serialization |
|`cirq.quil`| serialization  |
|`cirq.json_serialization`| serialization |


### Quantum operator representation protocols

The following family of protocols is an important and frequently used set of features of Cirq and it is worthwhile mentioning them and and how they interact with each other. They are, in the order of generality (starting with the least generic):

* `*unitary`
* `*mixture`
* `*channel`

All these protocols make it easier to work with different representations of quantum operators: finding that representation (`unitary`, `mixture`, `channel`), determining whether the operator has that representation (`has_*`) and applying them (`apply_*`) on a state vector. 

The `unitary` representation is the most special case, only applies when the quantum operator is unitary, and in this case represents it via the unitary matrix in the computational basis. We saw an example of the unitary protocol above.

The `mixture` representation is a probabilistic representation of a quantum operator, where each possible unitary effect $U_i$ occurs with a certain probability $p_i$, where $\sum p_i = 1$ In case the operator does not implement `SupportsMixture`, but does implement `SupportsUnitary`, `*mixture` protocols fall back to the `*unitary` methods, based on the simple correspondence: the $U$ is a mixture of a single unitary with probability $p=1$. 


In [ ]:
# cirq.Y has a unitary effect but does not implement SupportsMixture
print(cirq.mixture(cirq.Y))
print(cirq.has_mixture(cirq.Y))

The `channel` representation is the operator sum representation of a quantum operator (a channel):  
        $$
        \rho \rightarrow \sum_{k=0}^{r-1} A_k \rho A_k^\dagger
        $$
    These matrices are required to satisfy the trace preserving condition
        $$
        \sum_{k=0}^{r-1} A_k^\dagger A_k = I
        $$
    where $I$ is the identity matrix. The matrices $A_k$ are sometimes called Kraus or noise operators. 
    
In case the operator does not implement `SupportsChannel`, the `*channel` protocol will fall back to `*mixture` methods. 

In the simplest case of a simple unitary operator, `cirq.channel` returns the exact unitary matrix as `cirq.unitary`:

In [ ]:
print(cirq.channel(cirq.Y))
print(cirq.has_channel(cirq.Y))